# Tutorial 7: Training a Model

## Training a Sequence Labeling Model

In [ ]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from typing import List

# 1. get the corpus
corpus: TaggedCorpus = NLPTaskDataFetcher.load_corpus(NLPTask.CONLL_03).downsample(0.1)
print(corpus)

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

# 4. initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

# 8. plot training curves (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves('resources/taggers/example-ner/loss.tsv')
plotter.plot_weights('resources/taggers/example-ner/weights.txt')

### predict the class of new sentences.

In [ ]:
# load the model you trained
model = SequenceTagger.load_from_file('resources/taggers/example-ner/final-model.pt')

# create example sentence
sentence = Sentence('I love Berlin')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())

## Training a Text Classification Model


### corpus loading -- from Tutorial 6

In [34]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.data import TaggedCorpus
from pathlib import Path

# use your own data path
data_folder = Path('/home/wohlg/itmo/misc/cooking_classification/preprocessed')

# load corpus containing training, test and dev data
corpus = TaggedCorpus = NLPTaskDataFetcher.load_classification_corpus(data_folder,
                                                                     test_file='cooking.test',
                                                                     dev_file='cooking.valid',
                                                                     train_file='cooking.train').downsample(0.1)
    
print('Done loading')

## look at the corpus:


2019-04-20 21:31:27,556 Reading data from /home/wohlg/itmo/misc/cooking_classification/preprocessed
2019-04-20 21:31:27,559 Train: /home/wohlg/itmo/misc/cooking_classification/preprocessed/cooking.train
2019-04-20 21:31:27,560 Dev: /home/wohlg/itmo/misc/cooking_classification/preprocessed/cooking.valid
2019-04-20 21:31:27,562 Test: /home/wohlg/itmo/misc/cooking_classification/preprocessed/cooking.test
Done loading


In [20]:
label_dict = corpus.make_label_dictionary()

print(corpus.obtain_statistics())


{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1241,
        "number_of_documents_per_class": {
            "sauce": 39,
            "cheese": 24,
            "tea": 18,
            "food-safety": 92,
            "salmon": 6,
            "cookies": 22,
            "texture": 8,
            "american-cuisine": 1,
            "dehydrating": 3,
            "equipment": 84,
            "slow-cooking": 10,
            "crockpot": 5,
            "resources": 6,
            "storage-method": 34,
            "veal": 3,
            "free-range": 2,
            "oven": 26,
            "elderberries": 2,
            "kombucha": 1,
            "sorbet": 1,
            "seafood": 8,
            "bread": 52,
            "dough": 19,
            "yeast": 16,
            "ice-cream": 11,
            "pasta": 24,
            "beets": 3,
            "food-science": 22,
            "cooking-time": 12,
            "pressure-cooker": 3,
            "meat": 35,
           

In [35]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings, DocumentPoolEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer


# 1. get the corpus
# corpus: TaggedCorpus = NLPTaskDataFetcher.load_corpus(NLPTask.AG_NEWS, 'path/to/data/folder').downsample(0.1)

# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

# 3. make a list of word embeddings
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                   #FlairEmbeddings('news-forward'),
                   #FlairEmbeddings('news-backward'),
                   FlairEmbeddings('news-forward-fast'),
                   FlairEmbeddings('news-backward-fast'),
                   ]
word_embeddings = [CharacterEmbeddings()]
print('Done Here')


Done Here


In [36]:
print("OK")
word_embeddings

OK


[CharacterEmbeddings(
   (char_embedding): Embedding(275, 25)
   (char_rnn): LSTM(25, 25, bidirectional=True)
 )]

In [37]:
from flair.embeddings import CharacterEmbeddings

# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)

#document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
#                                                                     hidden_size=512,
#                                                                     reproject_words=True,
#                                                                     reproject_words_dimension=256,
#                                                                     )
    
print('Start')
document_embeddings= DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                     hidden_size=32,
                                                                     reproject_words=False,
                                                                     reproject_words_dimension=40,
                                                                     dropout=0.25
                                                                     )


    
#document_embeddings= DocumentPoolEmbeddings([CharacterEmbeddings()],
#                                              #flair_embedding_backward,
#                                              #flair_embedding_forward],
#                                              mode='max') # mean (default), min, max

# 5. create the text classifier
#classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False) 
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=True) 



Start


In [38]:
classifier




TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): CharacterEmbeddings(
        (char_embedding): Embedding(275, 25)
        (char_rnn): LSTM(25, 25, bidirectional=True)
      )
    )
    (word_reprojection_map): Linear(in_features=50, out_features=50, bias=True)
    (rnn): RNNBase(50, 32)
    (dropout): Dropout(p=0.25)
  )
  (decoder): Linear(in_features=32, out_features=512, bias=True)
  (loss_function): BCELoss()
)

In [39]:
# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus)

# 7. start the training
trainer.train('resources/taggers/cooking',
              learning_rate=0.2, # 0.1 
              mini_batch_size=8, # 32
              anneal_factor=0.5,
              patience=5,
              max_epochs=50) #150
                
# 8. plot training curves (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves('resources/taggers/cooking/loss.tsv')
plotter.plot_weights('resources/taggers/cooking/weights.txt')

2019-04-20 21:31:47,943 ----------------------------------------------------------------------------------------------------
2019-04-20 21:31:47,945 Evaluation method: MICRO_F1_SCORE
2019-04-20 21:31:47,948 ----------------------------------------------------------------------------------------------------
2019-04-20 21:31:48,032 epoch 1 - iter 0/156 - loss 0.08714703
2019-04-20 21:31:48,855 epoch 1 - iter 15/156 - loss 0.08703632
2019-04-20 21:31:49,794 epoch 1 - iter 30/156 - loss 0.08690077
2019-04-20 21:31:50,599 epoch 1 - iter 45/156 - loss 0.08676121
2019-04-20 21:31:51,384 epoch 1 - iter 60/156 - loss 0.08662141
2019-04-20 21:31:52,155 epoch 1 - iter 75/156 - loss 0.08647613
2019-04-20 21:31:52,973 epoch 1 - iter 90/156 - loss 0.08632807
2019-04-20 21:31:53,777 epoch 1 - iter 105/156 - loss 0.08617650
2019-04-20 21:31:54,730 epoch 1 - iter 120/156 - loss 0.08602074
2019-04-20 21:31:55,515 epoch 1 - iter 135/156 - loss 0.08586018
2019-04-20 21:31:56,275 epoch 1 - iter 150/156 - l

KeyboardInterrupt: 

In [40]:
classifier.state_dict

<bound method Module.state_dict of TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): CharacterEmbeddings(
        (char_embedding): Embedding(275, 25)
        (char_rnn): LSTM(25, 25, bidirectional=True)
      )
    )
    (word_reprojection_map): Linear(in_features=50, out_features=50, bias=True)
    (rnn): RNNBase(50, 32)
    (dropout): Dropout(p=0.25)
  )
  (decoder): Linear(in_features=32, out_features=512, bias=True)
  (loss_function): BCELoss()
)>

In [13]:
from flair.data import Sentence

classifier = TextClassifier.load_from_file('resources/taggers/cooking/final-model.pt')

# create example sentence
sentence = Sentence('where is the bacon is it in the oven.')

# predict tags and print
classifier.predict(sentence)

print(sentence.labels)

2019-04-20 21:24:50,885 loading file resources/taggers/cooking/final-model.pt
[]
